# Assignment — Introduction to Network Science

In this assignment, we will consider some basics of networkx 2.5.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import requests
from zlib import adler32

In [ ]:
assert nx.__version__ > '2.5'

In case of an exception, install networkx 2.5.1 using `pip` as follows:
```
pip uninstall networkx
pip install networkx==2.5.1
```

### Task 1. Reading a graph from a file (1 point)

In [ ]:
url = 'https://raw.githubusercontent.com/netspractice/network-science/main/datasets/the_lord_of_the_rings_adjlist.txt'
open('the_lord_of_the_rings_adjlist.txt', 'wb').write(requests.get(url).content)

url = 'https://raw.githubusercontent.com/netspractice/network-science/main/datasets/the_lord_of_the_rings_edgelist.txt'
open('the_lord_of_the_rings_edgelist.txt', 'wb').write(requests.get(url).content);


A common representation of a graph is an adjacency matrix, but many real-world graphs are sparse (many zeros in an adjacency matrix), so in most cases data is stored in cheap sparse formats
1. edge list
2. adjacency list

Edge list is a list of pairs of nodes — a target and destination of edges. Here is an edge list of an interaction network of films "The Lord of the Rings" (2001-2003).

Edges are the links
Nodes are the actors or vertices

In [ ]:
with open('the_lord_of_the_rings_edgelist.txt') as f:
    print(f.read()[:])

Adjacency list is a list of lists of nodes where the first node is a root and others are leafs

In [ ]:
with open('the_lord_of_the_rings_adjlist.txt') as f:
    print(f.read()[:])

For most cases we can use given functions in networkx:

* `nx.read_adjlist`
* `nx.read_edgelist`
* `nx.read_weighted_edgelist`

In the further task, read graphs `the_lord_of_the_rings_edgelist.txt`, `the_lord_of_the_rings_adjlist.txt` and combine them into a single graph such that the same nodes are merged in a single node preserving all adjacent edges.

*Hint: use `graph.add_edges_from`*

In [ ]:
def read_and_merge_graphs(edgelist_path, adjlist_path, delimiter):
    adjlist = nx.read_adjlist(adjlist_path, delimiter=delimiter) # Start is the main node and everything after are leaves
    edgelist = nx.read_edgelist(edgelist_path, delimiter=delimiter) # Just bidrectional connections on each row
    
    G = nx.compose(adjlist, edgelist)
    return G

In [ ]:
G = read_and_merge_graphs("the_lord_of_the_rings_edgelist.txt", 
                          "the_lord_of_the_rings_adjlist.txt", delimiter=", ")
assert set(G.neighbors('SAM')) == {
    'BILBO',
    'ELROND',
    'FRODO',
    'GALADRIEL',
    'GANDALF',
    'GOLLUM',
    'GORBAG',
    'SHAGRAT',
    'SMEAGOL'}
assert nx.number_of_nodes(G) == 35
assert nx.number_of_edges(G) == 98

### Task 2. Nodes labeling (1 point)

Nodes in many real-world graphs represents some objects, say people in social networks or atoms in molecular networks. Usually, names of such nodes is in the string format, but in many cases it can be useful to make names and indices of nodes equal. Using such labeling, we can refer to a particular node by its index.

Write a function `convert_node_names_to_int` that takes a graph with string names and returns a new graph with integer names so that first node in the initial graph is called 0 in the new graph, second is 1 and so on. Also, save initial names in the node attribute `name`.

AP: Often it is useful to just **label the nodes with their respective index** so that it is easier afterwards to call and anal
yze them.

In [ ]:
def convert_node_names_to_int(G):
    mapping = dict(zip(list(G.nodes()),range(len(list(G.nodes())))))
    newG = nx.relabel_nodes(G, mapping)
    attributes = {}
    for x in range(0, len(list(G.nodes))):
        old_name = list(G.nodes)[x]
        attributes.update({x: {"name": old_name}})

    nx.set_node_attributes(newG, attributes)
    return newG

In [ ]:
newG = convert_node_names_to_int(G)
for i in range(len(G)):
    assert list(G.nodes)[i] == newG.nodes[i]['name']
assert np.allclose(nx.to_numpy_array(G), nx.to_numpy_array(newG))

### Task 3. Directed graph (1 point)

The direction of edges could be crucial feature for many tasks, e.g. link analysis. Directed edges are also known as links or arcs. In networkx, the type of graph is defined by a constructor: `nx.Graph` for undirected graph, `nx.DiGraph` for directed graph. A directed graph can have an asymmetric adjacency matrix.

In [46]:
diG = nx.DiGraph()
diG.add_edges_from([[0, 1], [1, 2]])
nx.to_numpy_array(diG)

#nx.draw(diG)

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.]])

Write a function `convert_to_digraph` that takes an undirected graph and returns a new directed graph where the direction is defined by the rule: the node $i$ follows to $j$ if the length of `name` of $i$ is at least the length of `name` of $j$. Keep the ordering of nodes and the node attribute `name`.

In [ ]:
def convert_to_digraph(G):
    # YOUR CODE HERE
    raise NotImplementedError()

In [66]:
newGG = nx.DiGraph()
newGG.add_nodes_from(newG)
newGG.nodes
assert list(newGG.nodes) == list(newG.nodes) #ok this passes

list(newG.nodes)
 
container_list = []
for i in range(0, len(newG.nodes)):
    for j in range(0, len(newG.nodes)):
        check_longer = len(newG.nodes[i]['name'])>= len(newG.nodes[j]['name'])
        check_existing = nx.to_numpy_matrix(newG)[i,j] == 1
        if check_longer and check_existing:
            container_list.append([i, j])

117

In [ ]:
diG = convert_to_digraph(newG)
assert list(diG.nodes) == list(newG.nodes)
for i, j in diG.edges:
    assert len(diG.nodes[i]['name']) >= len(diG.nodes[j]['name'])
assert len(diG.edges) == 117

### Task 4. Basic statistics (1 point)

Networkx provides a simple interface for getting basic graph statistics (look at the documentation to recall definitions):
* `nx.radius`
* `nx.diameter`
* `nx.average_shortest_path_length`
* `nx.average_clustering`

For example, here are statistics of a path graph with 3 nodes:

In [ ]:
G = nx.path_graph(3)
print('Radius: ', nx.radius(G))
print('Diameter: ', nx.diameter(G))
print('Average shortest path length: ', nx.average_shortest_path_length(G))
print('Average clustering coefficient: ', nx.average_clustering(G))
nx.draw_networkx(G)

Your task is to create graphs by given statistics. We can create a graph using simple networkx interfaces. For example, we can create a graph by passing an edgelist.

In [ ]:
G = nx.Graph([[0, 1], [1, 2], [2, 0]])
nx.draw_networkx(G)

Or create an empty graph and then add edges one by one.

In [ ]:
G = nx.Graph()
G.add_edge(0, 1)
G.add_edge(1, 2)
G.add_edge(2, 0)
nx.draw_networkx(G)

Your task is to create graphs by given statistics.

Write a function `graph1` that returns a graph with a radius 3 and diameter 3.

In [ ]:
def graph1():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = graph1()
assert nx.radius(G) == nx.diameter(G) == 3
nx.draw_networkx(G)

Write a function `graph2` that returns a graph with a diameter of at least 5 average shortest path lengths.

In [ ]:
def graph2():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = graph2()
assert nx.diameter(G) >= 5 * nx.average_shortest_path_length(G)
nx.draw_networkx(G)

Write a function `graph3` that returns a graph with an average clustering coefficient between 0.1 and 0.11.

In [ ]:
def graph3():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = graph3()
assert 0.1 <= nx.average_clustering(G) <= 0.11
nx.draw_networkx(G)

### Task 5. Graph layout (2 points)

A graph layout is a dictionary with a location of every node: keys are nodes and values are x, y positions. By default in `nx.draw_networkx`, it is the spring layout — a force-directed representation of the network treating edges as springs holding nodes close, while treating nodes as repelling objects.

In [ ]:
G = nx.barabasi_albert_graph(10, 2)
layout = nx.spring_layout(G)
layout

In [ ]:
nx.draw_networkx(G, pos=layout)

The Kamada Kawai layout is based on path-length cost-function. Sometimes, it is quite similar to the spring layout, however a significant difference is that the spring layout is random and the Kamada Kawai is deterministic.

In [ ]:
layout = nx.kamada_kawai_layout(G)
nx.draw_networkx(G, pos=layout)

Circular layout allows to draw a graph where nodes are in a circle.

In [ ]:
layout = nx.circular_layout(G)
plt.figure(figsize=(5, 5))
nx.draw_networkx(G, pos=layout)

Bipartite layout allows to draw bipartite graph.

In [ ]:
B = nx.Graph()
B.add_nodes_from([1, 2, 3, 4], bipartite=0)
B.add_nodes_from(["a", "b", "c"], bipartite=1)
B.add_edges_from([(1, "a"), (1, "b"), (2, "b"), (2, "c"), (3, "c"), (4, "a")])
layout = nx.bipartite_layout(B, [1, 2, 3, 4])
nx.draw_networkx(B, pos=layout)

There are other layouts in networkx, also there are some useful parameters that you can find in documentation. Next, let us visualize a coappearance network of characters in The Matrix — a 1999 American science fiction action film written and directed by the Wachowskis. The dataset is taken from https://moviegalaxies.com/.

In [ ]:
url = 'https://raw.githubusercontent.com/network-science-course/network-science-course/main/datasets/the_matrix_graph.gexf'
open('the_matrix_graph.gexf', 'wb').write(requests.get(url).content);

In [ ]:
def get_matrix_graph():
    G = nx.read_gexf('the_matrix_graph.gexf')
    mapping = {}
    for node in G.nodes:
        mapping[node] = G.nodes[node]['label']
    G = nx.relabel_nodes(G, mapping)
    return G
matrixG = get_matrix_graph()
plt.figure(figsize=(13, 7))
layout = nx.kamada_kawai_layout(matrixG)
nx.draw_networkx(matrixG, pos=layout)

Your task is to draw the matrix graph using the kamada kawai layout with nonlinear modification to obtain a denser layout away from the NEO node. Let us define a proximity coefficient using Gaussian kernel

$$\text{coef}(i) = \exp\left(-\frac{\text{distance}^2}{2\sigma^2}\right)$$

where a _distance_ is the Euclidean distance from the node $i$ to the NEO node, $\sigma$ is the constant. Then the nodes should be moved towards the NEO node such that

$$\text{distance} = \text{coef}(i) \times \text{distance}$$

The simplest way to do it is to center the layout so that the NEO node is in x=0 y=0 and then to multiply all node positions by the proximity coefficient.

Write a function `dense_neo_layout` that takes a layout, sigma and returns a new (dense) layout.

_Hint: use `np.linalg.norm` to calculate Euclidean norm._

In [ ]:
def dense_neo_layout(layout, sigma):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
layout = nx.kamada_kawai_layout(matrixG)
layout = dense_neo_layout(layout, sigma=1.5)
assert type(layout) == dict
assert np.all(layout['NEO'] == [0, 0])
assert np.all(layout['REX'].round(2) == [-0.67,  0.58])
assert adler32(str(layout['AGENT BROWN'].round(2)[0]).encode()) == 32309448

In [ ]:
plt.figure(figsize=(13, 7))
nx.draw_networkx(matrixG, layout)

### Task 6. Nodes visualization (1 point)

We can set a node size using `draw_networkx` with a parameter `node_size`. It should be a list of values ordered by `G.nodes`. For example, consider a simple path graph.

In [ ]:
G = nx.path_graph(3)
G.nodes

Let us set a size 200 for the node 0, 300 for 1, 400 for 2.

In [ ]:
layout = nx.kamada_kawai_layout(G)
nx.draw_networkx(G, pos=layout, node_size=[200, 300, 400])

To color nodes, we can use `node_color`. It should be a list of colors:
* a tuple of float RGB 0-1 values: (red, green, blue)
* a tuple of float RGBA 0-1 values: (red, green, blue, alpha)
* a string — color name

In [ ]:
nx.draw_networkx(
    G,
    pos=layout,
    node_color=[(0.4, 0.8, 0, 0.6), (0.7, 0.4, 1, 0.6), (1, 0, 0.5, 0.6)])

In [ ]:
nx.draw_networkx(
    G,
    pos=layout,
    node_color=['green', 'cyan', 'pink'])

Also we can use a colors from matplotlib colormaps. Discrete:

In [ ]:
colors = plt.cm.Set1.colors
nx.draw_networkx(
    G,
    pos=layout,
    node_color=[colors[0], colors[1], colors[2]])

Or continuous:

In [ ]:
rainbow = plt.cm.rainbow
nx.draw_networkx(
    G,
    pos=layout,
    node_color=[rainbow(0.2), rainbow(0.8), rainbow(0.9)])

Another option of coloring nodes is to pass a list of single values by the `node_color` and colormap by the `cmap` parameter. In this case, the lowest value will assigned to the first color in colormap and vise versa.

In [ ]:
nx.draw_networkx(
    G,
    pos=layout,
    cmap = plt.cm.hot,
    node_color=[1, 2, 3])

We can forcibly set a color range using `vmin` and `vmax`.

In [ ]:
nx.draw_networkx(
    G,
    pos=layout,
    cmap = plt.cm.hot,
    node_color=[1, 2, 3],
    vmin=0, vmax=4)

We can border nodes using `edgecolors` and `linewidths` parameters.

In [ ]:
nx.draw_networkx(
    G=G, 
    pos=layout,
    cmap = plt.cm.hot, 
    node_color=[1, 2, 3],
    vmin=0, vmax=4, 
    edgecolors='black', 
    linewidths=1)

Your task is to adjust a size and color of nodes in the matrix graph, size and color are taken from a node degree and node attribute `G.nodes[node]`.

In [ ]:
matrixG.nodes['NEO']

Write a function `drawing_nodes_params` that takes a graph and returns a dictionary with parameters for `nx.draw_networkx` of the form `{'G': graph, 'node_color': color, ...}`. Parameters should contain a graph, kamada kawai layout, node colors from `viz.color` attribute. The colors should be converted from RGB range 0-255 to RGB range 0-1. The node size is a node degree `G.degree[node]` multiplied by 100.

In [ ]:
def drawing_nodes_params(graph):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
matrixG = get_matrix_graph()
params = drawing_nodes_params(matrixG)
assert type(params) == dict
keys = ['G', 'pos', 'node_color', 'node_size']
assert len(set(params.keys()).intersection(keys)) == 4
assert np.all(params['node_color'][10].round(2) == [0.37, 0.79, 0.75])
assert adler32(str(params['node_color'][0][1].round(2)).encode()) == 19071123
assert params['node_size'][0] == 200
assert params['node_size'][1] == 600
assert nx.to_numpy_array(params['G']).sum() == 326.0

In [ ]:
plt.figure(figsize=(13, 7))
nx.draw_networkx(**params )

Sometimes, it will be useful to draw nodes in nonlinear scale. If we want to make node sizes more comparable, we can use polynomial or exponential scale. On the other hand, we can use logarithmic scale to make sizes more similar.

Write a function `drawing_nodes_polyparams` that returns the same dictionary as the `drawing_nodes_params` function, but the node size is a sum of node degree in a power 3 and 10.

$$\text{size} = \text{degree}^3 + 10$$

In [ ]:
def drawing_nodes_polyparams(graph):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
matrixG = get_matrix_graph()
params = drawing_nodes_polyparams(matrixG)
assert type(params) == dict
keys = ['G', 'pos', 'node_color', 'node_size']
assert len(set(params.keys()).intersection(keys)) == 4
assert np.all(params['node_color'][10].round(2) == [0.37, 0.79, 0.75])
assert adler32(str(params['node_color'][0][1].round(2)).encode()) == 19071123
assert params['node_size'][0] == 18
assert params['node_size'][1] == 226
assert nx.to_numpy_array(params['G']).sum() == 326.0

In [ ]:
plt.figure(figsize=(13, 7))
nx.draw_networkx(**params )

### Task 7. Edges visualization, weighted graphs (1 point)

We can set an edge width using `draw_networkx` with a parameter `width`. It should be a list of values sorted by G.edges. For example, consider a simple path graph.

In [ ]:
G = nx.path_graph(4)
G.edges

Let us set a width 1 for the edge 0-1, 5 for 1-2, 10 for 2-3.

In [ ]:
layout = nx.kamada_kawai_layout(G)
nx.draw_networkx(G, pos=layout, width=[1, 5, 10])

If we create a graph by a weighted ajacency matrix, weights will be stored in an edge attribute `weight`.

In [ ]:
W = np.array([[0, 1, 0, 0], 
              [1, 0, 5, 0], 
              [0, 5, 0, 10], 
              [0, 0, 10, 0]])
G = nx.from_numpy_array(W)

In this case, we can get a value using `G.edges[edge]` view

In [ ]:
G.edges[(0, 1)]

Or we can get all values using `nx.get_edge_attributes`

In [ ]:
nx.get_edge_attributes(G, 'weight')

In [ ]:
weights = list(nx.get_edge_attributes(G, 'weight').values())
nx.draw_networkx(G, pos=layout, width=weights)

If we want to apply specific color or style to some edges, it is possible by `edge_color` and `style` parameters. For example, we can draw an edge with a weight 10 as a red dotted line.

In [ ]:
nx.draw_networkx(
    G, pos=layout, 
    width=[1, 5, 10], 
    edge_color=['black', 'black', 'red'], 
    style=['solid', 'solid', 'dotted'])

Your task is to draw the matrix graph where an edge width is 0 if a weight is 1, otherwise it is a weight value itself. An edge color is `red` if weight is at least 5 and `black` otherwise.

In [ ]:
def drawing_edges_params(graph):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
matrixG = get_matrix_graph()
params = drawing_edges_params(matrixG)
assert type(params) == dict
keys = ['G', 'pos', 'edge_color', 'width']
assert len(set(params.keys()).intersection(keys)) == 4
assert nx.to_numpy_array(params['G']).sum() == 326.0
params['width'][0] == 2.0
assert np.array(params['width']) @ params['width'] == 508
assert adler32(str(params['edge_color']).encode()) == 646700613

In [ ]:
plt.figure(figsize=(13, 7))
nx.draw_networkx(**params)

### Task 8. Labels visualization (1 point)

In this task, we will modify some label parameters: font size, position, visibility. In networkx, we cannot pass a list of font sizes to `draw_networkx`, so we will use drawing by layers using other methods:
* `draw_networkx_nodes` — draw the nodes only
* `draw_networkx_edges` — draw the edges only
* `draw_networkx_labels` — draw the node labels only
* `nx.draw_networkx_edge_labels` — draw the edge labels only

Also we can separately adjust an `alpha` value for nodes, edges, labels. Here is a simple example, let us draw a path graph with 4 nodes where the node 0 is labeled as "a", 3 is "b", the edge 1-2 is labeled as "edge", and edges are transparent with `alpha=0.2`.

In [ ]:
G = nx.path_graph(4)
layout = nx.kamada_kawai_layout(G)
nx.draw_networkx_nodes(G, pos=layout)
nx.draw_networkx_edges(G, pos=layout, alpha=0.2)
nx.draw_networkx_labels(G, pos=layout, labels={0: 'a', 3: 'b'})
nx.draw_networkx_edge_labels(G, pos=layout, edge_labels={(1, 2): 'edge'})
plt.show()

If we want to adjust a font size, it is possible by repeated drawing labels with different `font_size` values. For example:

In [ ]:
G = nx.path_graph(4)
layout = nx.kamada_kawai_layout(G)
nx.draw_networkx_nodes(G, pos=layout)
nx.draw_networkx_edges(G, pos=layout, alpha=0.2)
nx.draw_networkx_labels(G, pos=layout, labels={3: 'b'}, font_size=10)
nx.draw_networkx_labels(G, pos=layout, labels={0: 'a'}, font_size=20)
nx.draw_networkx_edge_labels(G, pos=layout, edge_labels={(1, 2): 'edge'})
plt.show()

Your task is to draw the matrix graph with settings: 
* labels are invisible if a node degree less than 6
* a font size is a node degree
* all labels are slided down so that $y = y - 0.025 \sqrt{\text{degree}}$

Write a function `labels_list_parameters` that takes a graph, layout and returns a list of parameters for repited drawing labels using `nx.draw_networkx_labels`. Each element of the list is a dictionary that can be passed to `nx.draw_networkx_labels`. The first dictionary is for the node degree 6, the second is for 7 and so on.

In [ ]:
def labels_list_parameters(graph, layout):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
matrixG = get_matrix_graph()
layout = nx.kamada_kawai_layout(matrixG)
list_params = labels_list_parameters(matrixG, layout)
assert type(list_params) == list
assert type(list_params[0]) == dict
assert len(list_params) == 8
assert nx.to_numpy_array(list_params[0]['G']).sum() == 326
assert list_params[0]['font_size'] == 6
assert adler32(str(np.array(list_params[7]['font_size'])).encode()) == 10027110
assert np.all(list_params[0]['pos']['MAN'].round(2) == [-0.16, 0.05])
assert adler32(
    str(list_params[0]['pos']['AGENT JONES'].round(2)[1]).encode()) == 46465263
assert adler32(
    str(list_params[7]['pos']['NEO'].round(2)[1]).encode()) == 46792946
assert list_params[1]['labels']['AGENT SMITH'] == 'AGENT SMITH'
assert list_params[1]['labels'].get('AGENT JONES') is None \
    or list_params[1]['labels']['AGENT JONES'] == ''
assert len([val for val in  list_params[2]['labels'].values() if val != '']) == 3

In [ ]:
plt.figure(figsize=(13, 7))
layout = nx.kamada_kawai_layout(matrixG)
nx.draw_networkx_nodes(matrixG, pos=layout)
nx.draw_networkx_edges(matrixG, pos=layout, alpha=0.2)
for params in labels_list_parameters(matrixG, layout):
    nx.draw_networkx_labels(**params)
plt.show()

### Task 9. Visible area (1 points)

Drawing a large graph, it may be useful to zoom in an area near some nodes by `plt.xlim(x_min, x_max)` and `plt.ylim(y_min, y_max)` where `x_max` is a right border, `y_max` is a top border. 

In [ ]:
G = nx.path_graph(4)
layout = nx.kamada_kawai_layout(G)
nx.draw_networkx(G, pos=layout)
plt.xlim(-0.5, 0.5)
plt.ylim(-0.5, 0.5)
plt.show()

Let us adjust a visible area and combine previous drawing techniques to get a more meaningful figure:
* take a node color from the `viz.color` node attribute converting 0-255 range to 0-1 range
* set a node size as $5\times\text{degree}^2$
* take an edge width from the `weight` edge attribute
* set an edge alpha is 0.2
* labels are invisible if a node degree less than 6
* set a font size as $3 \times \text{degree}^{2/3}$
* set a visible area such that a lower border is NEO, higher is AGENT SMITH, left is MORPHEUS, right is TANK.

Write a function `get_all_params` that takes a graph and layout and returns a dictionary with keys:
* `nodes_params` — dict, parameters for `nx.draw_networkx_nodes`
* `edges_params` — dict, parameters for `nx.draw_networkx_edges`
* `labels_param_list` — list of dict, parameters for `nx.draw_networkx_edges`, for degrees 6, 7 and so on
* `area` — list of float vlues: a left, right, lower, higher borders of the visible area

In [ ]:
def get_all_params(graph, layout):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
matrixG = get_matrix_graph()
layout = nx.kamada_kawai_layout(matrixG)
all_params = get_all_params(matrixG, layout)
assert type(all_params) == dict
keys = ['nodes_params', 'edges_params', 'labels_param_list', 'area']
assert len(set(all_params.keys()).intersection(keys)) == 4
assert nx.to_numpy_array(all_params['nodes_params']['G']).sum() == 326
assert np.all(all_params['nodes_params']['pos']['NEO'].round(2) == [0., -0.3])
assert np.all(all_params['nodes_params']['node_color'][4].round(2) == [1., 0.86, 0.4])
assert all_params['nodes_params']['node_size'][10] == 20
assert nx.to_numpy_array(all_params['edges_params']['G']).sum() == 326
assert np.all(all_params['edges_params']['pos']['NEO'].round(2) == [0. , -0.3])
assert all_params['edges_params']['alpha'] == 0.2
assert all_params['edges_params']['width'][20] == 2
assert type(all_params['labels_param_list']) == list
assert nx.to_numpy_array(all_params['labels_param_list'][0]['G']).sum() == 326
assert np.all(all_params['labels_param_list'][5]['pos']['NEO'].round(2) == [0. , -0.3])
assert all_params['labels_param_list'][5]['labels'].get('WOMAN') is None \
    or all_params['labels_param_list'][5]['labels']['WOMAN'] == ''
assert round(all_params['labels_param_list'][2]['font_size'], 4) == 12
assert all_params['area'][0] < all_params['area'][1]
assert all_params['area'][2] < all_params['area'][3]

In [ ]:
plt.figure(figsize=(10, 7))
nx.draw_networkx_nodes(**all_params['nodes_params'])
nx.draw_networkx_edges(**all_params['edges_params'])
for params in all_params['labels_param_list']:
    nx.draw_networkx_labels(**params)
plt.xlim(all_params['area'][0] - 0.15, all_params['area'][1] + 0.1)
plt.ylim(all_params['area'][2] - 0.10, all_params['area'][3] + 0.1)
plt.show()